# ETL Exploration & Cleaning — Flight Sample Data (10 days)

**Goal**: Explore CSVs, validate column consistency, clean data, and export JSONL for MongoDB.

> This notebook mirrors the production ETL (`etl/etl_flights.py`) but lets you iterate interactively.
> Once you're happy with the results here, move the final logic back to your script for reproducibility.


In [31]:
# 🚧 Parameters (adjust for your Mac project paths) 

from pathlib import Path

# Project-relative paths (recommended repo structure)
RAW_DIR = Path("../data_raw")        # folder with daily CSVs: flight_sample_YYYY-MM-DD.csv
OUT_DIR = Path("../data_enriched")   # where JSONL exports will be written

# File pattern for 10 days (adjust if needed)
GLOB_PATTERN = "flight_sample_2022-09-0*.csv"   # e.g., 2022-09-01 ... 2022-09-10

RAW_DIR, OUT_DIR, GLOB_PATTERN

(PosixPath('../data_raw'),
 PosixPath('../data_enriched'),
 'flight_sample_2022-09-0*.csv')

In [32]:
# 📦 Imports
import pandas as pd
from datetime import datetime, timezone
import json
import numpy as np

OUT_DIR.mkdir(exist_ok=True, parents=True)
pd.set_option("display.max_columns", 50)

## 1) Load one day and preview (sanity check)

In [33]:
sample_path = next(RAW_DIR.glob("flight_sample_2022-09-01.csv"))
df_sample = pd.read_csv(sample_path)
print("Raw columns:", list(df_sample.columns))
df_sample.head(10)

Raw columns: ['icao24', 'firstseen', 'lastseen', 'callsign', 'estdepartureairport', 'estarrivalairport', 'model', ' typecode', ' registration']


,icao24,firstseen,lastseen,callsign,estdepartureairport,estarrivalairport,model,typecode,registration
0,44cdc6,1.662054e+09,1662065618,BEL40V,NaN,EBBR,A320 214,A320,OO-SNF
1,44cdc6,1.662032e+09,1662047517,BEL7QJ,EBBR,NaN,A320 214,A320,OO-SNF
2,44cdc6,1.662022e+09,1662028405,BEL8DG,LIRF,EBBR,A320 214,A320,OO-SNF
3,44cdc6,1.662011e+09,1662016737,BEL1YC,EBBR,LIRF,A320 214,A320,OO-SNF
4,4601f7,1.662067e+09,1662068749,FIN7A,EFHK,EFTP,ATR 72 500,AT75,OH-ATJ
5,4601f7,1.662059e+09,1662060352,FIN5AZ,EETN,EFHK,ATR 72 500,AT75,OH-ATJ
6,4601f7,1.662056e+09,1662057089,FIN9D,EFHK,EETN,ATR 72 500,AT75,OH-ATJ
7,4601f7,1.662052e+09,1662053667,FIN27H,EETN,EFHK,ATR 72 500,AT75,OH-ATJ
8,4601f7,1.662050e+09,1662050774,FIN5Y,EFHK,EETN,ATR 72 500,AT75,OH-ATJ
9,4601f7,1.662046e+09,1662047514,FIN3ZP,NaN,EFHK,ATR 72 500,AT75,OH-ATJ


In [34]:
# Some columns in the provided CSV have leading spaces (e.g., ' typecode', ' registration').
# Standardize by stripping whitespace from all column names.
df_sample.columns = [c.strip() for c in df_sample.columns]
print("Stripped columns:", list(df_sample.columns))
df_sample.head(10)

Stripped columns: ['icao24', 'firstseen', 'lastseen', 'callsign', 'estdepartureairport', 'estarrivalairport', 'model', 'typecode', 'registration']


,icao24,firstseen,lastseen,callsign,estdepartureairport,estarrivalairport,model,typecode,registration
0,44cdc6,1.662054e+09,1662065618,BEL40V,NaN,EBBR,A320 214,A320,OO-SNF
1,44cdc6,1.662032e+09,1662047517,BEL7QJ,EBBR,NaN,A320 214,A320,OO-SNF
2,44cdc6,1.662022e+09,1662028405,BEL8DG,LIRF,EBBR,A320 214,A320,OO-SNF
3,44cdc6,1.662011e+09,1662016737,BEL1YC,EBBR,LIRF,A320 214,A320,OO-SNF
4,4601f7,1.662067e+09,1662068749,FIN7A,EFHK,EFTP,ATR 72 500,AT75,OH-ATJ
5,4601f7,1.662059e+09,1662060352,FIN5AZ,EETN,EFHK,ATR 72 500,AT75,OH-ATJ
6,4601f7,1.662056e+09,1662057089,FIN9D,EFHK,EETN,ATR 72 500,AT75,OH-ATJ
7,4601f7,1.662052e+09,1662053667,FIN27H,EETN,EFHK,ATR 72 500,AT75,OH-ATJ
8,4601f7,1.662050e+09,1662050774,FIN5Y,EFHK,EETN,ATR 72 500,AT75,OH-ATJ
9,4601f7,1.662046e+09,1662047514,FIN3ZP,NaN,EFHK,ATR 72 500,AT75,OH-ATJ


## 2) Profile & nulls

In [35]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115725 entries, 0 to 115724
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   icao24               115725 non-null  object 
 1   firstseen            115571 non-null  float64
 2   lastseen             115725 non-null  int64  
 3   callsign             115541 non-null  object 
 4   estdepartureairport  85973 non-null   object 
 5   estarrivalairport    97105 non-null   object 
 6   model                107589 non-null  object 
 7   typecode             79348 non-null   object 
 8   registration         108923 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 7.9+ MB


In [36]:
df_sample.isna().sum().sort_values(ascending=False)

typecode               36377
estdepartureairport    29752
estarrivalairport      18620
model                   8136
registration            6802
callsign                 184
firstseen                154
icao24                     0
lastseen                   0
dtype: int64

## 3) Quick value checks (airports, callsigns)

In [37]:
print("Unique dep airports (sample):", df_sample['estdepartureairport'].astype(str).str.upper().unique()[:20])
print("Unique arr airports (sample):", df_sample['estarrivalairport'].astype(str).str.upper().unique()[:20])
print("Calls sign examples:", df_sample['callsign'].astype(str).str.upper().dropna().unique()[:20])

Unique dep airports (sample): ['NAN' 'EBBR' 'LIRF' 'EFHK' 'EETN' 'ESGG' 'EFTU' 'ENGM' 'EKCH' 'ENFL'
 'EGGP' 'LIPE' 'EDDF' 'LPPT' 'EPKT' 'EDLW' 'LGAV' 'EPNT' 'LKTB' 'LKHK']
Unique arr airports (sample): ['EBBR' 'NAN' 'LIRF' 'EFTP' 'EFHK' 'EETN' 'ESGG' 'EKCH' 'ENGM' 'ENFL'
 'EGNR' 'LIMC' 'LIPE' 'LAGJ' 'LPPT' 'EDDF' 'EPKT' 'EDLW' 'LGAV' 'EPNT']
Calls sign examples: ['BEL40V  ' 'BEL7QJ  ' 'BEL8DG  ' 'BEL1YC  ' 'FIN7A   ' 'FIN5AZ  '
 'FIN9D   ' 'FIN27H  ' 'FIN5Y   ' 'FIN3ZP  ' 'FIN1TJ  ' 'FIN5AJ  '
 'FIN863  ' 'FIN5TB  ' 'FIN6BL  ' 'FIN36U  ' 'FIN1860 ' 'FIN65P  '
 'FIN1DP  ' 'FIN1AM  ']


## 4) Helper functions for cleaning

In [38]:
import re
from datetime import datetime, timezone

ICAO4 = re.compile(r"^[A-Z0-9]{4}$")  # strict ICAO code pattern

def to_dt(ts):
    """UNIX seconds -> aware UTC datetime; None if invalid/NaN."""
    if ts is None:
        return None
    try:
        # pandas NaN is float('nan'), pd.isna handles both None and NaN
        import pandas as pd
        if pd.isna(ts):
            return None
    except Exception:
        pass
    try:
        return datetime.fromtimestamp(int(ts), tz=timezone.utc)
    except Exception:
        return None

def clean_callsign(x):
    """Uppercase trimmed callsign; None if empty/NaN."""
    import pandas as pd
    if pd.isna(x):
        return None
    s = str(x).strip().upper()
    return s if len(s) > 0 else None

def clean_icao_airport(x):
    """
    Return 4-char ICAO (uppercase) or None.
    Never returns 'NAN' (pandas NaN coerced to string) or 3-letter IATA.
    """
    import pandas as pd
    if pd.isna(x):
        return None
    s = str(x).strip().upper()
    return s if ICAO4.fullmatch(s) else None


## 5) Load ALL days and concatenate

In [39]:
files = sorted(RAW_DIR.glob(GLOB_PATTERN))[:10]  # take first 10 matches
print("Found files:", [p.name for p in files])
df = pd.concat((pd.read_csv(p) for p in files), ignore_index=True)
print("Raw columns:", list(df.columns))
df.columns = [c.strip() for c in df.columns]
print("Stripped columns:", list(df.columns))
df.head(5)

Found files: ['flight_sample_2022-09-01.csv', 'flight_sample_2022-09-02.csv', 'flight_sample_2022-09-03.csv', 'flight_sample_2022-09-04.csv', 'flight_sample_2022-09-05.csv', 'flight_sample_2022-09-06.csv', 'flight_sample_2022-09-07.csv', 'flight_sample_2022-09-08.csv', 'flight_sample_2022-09-09.csv']
Raw columns: ['icao24', 'firstseen', 'lastseen', 'callsign', 'estdepartureairport', 'estarrivalairport', 'model', ' typecode', ' registration']
Stripped columns: ['icao24', 'firstseen', 'lastseen', 'callsign', 'estdepartureairport', 'estarrivalairport', 'model', 'typecode', 'registration']


,icao24,firstseen,lastseen,callsign,estdepartureairport,estarrivalairport,model,typecode,registration
0,44cdc6,1.662054e+09,1662065618,BEL40V,NaN,EBBR,A320 214,A320,OO-SNF
1,44cdc6,1.662032e+09,1662047517,BEL7QJ,EBBR,NaN,A320 214,A320,OO-SNF
2,44cdc6,1.662022e+09,1662028405,BEL8DG,LIRF,EBBR,A320 214,A320,OO-SNF
3,44cdc6,1.662011e+09,1662016737,BEL1YC,EBBR,LIRF,A320 214,A320,OO-SNF
4,4601f7,1.662067e+09,1662068749,FIN7A,EFHK,EFTP,ATR 72 500,AT75,OH-ATJ


## 6) Normalize & select columns

In [40]:
import pandas as pd
import numpy as np

want = [
    "icao24","firstseen","lastseen","callsign",
    "estdepartureairport","estarrivalairport",
    "model","typecode","registration"
]
present = [c for c in want if c in df.columns]
df = df[present].copy()

# Ensure metadata columns are string or None (never NaN)
for col in ["model", "typecode", "registration"]:
    if col in df.columns:
        # keep strings as-is, convert NaN to None, strip whitespace
        df[col] = (
            df[col]
            .apply(lambda v: None if pd.isna(v) else str(v).strip())
            .replace({"": None})
        )

# icao24 as lowercase hex string, None if missing
df["icao24"] = df["icao24"].apply(lambda v: str(v).lower() if pd.notna(v) else None)

# callsign cleaned (None if empty)
df["callsign"] = df["callsign"].apply(clean_callsign)

# airports — DO NOT astype(str); validate ICAO
df["estdepartureairport"] = df["estdepartureairport"].apply(clean_icao_airport)
df["estarrivalairport"]   = df["estarrivalairport"].apply(clean_icao_airport)

# timestamps -> datetime
df["departure_time"] = df["firstseen"].apply(to_dt) if "firstseen" in df else None
df["arrival_time"]   = df["lastseen"].apply(to_dt)  if "lastseen"  in df else None

# duration (minutes) only if both present
if {"firstseen","lastseen"}.issubset(df.columns):
    dur = (df["lastseen"] - df["firstseen"]) / 60.0
    df["duration_min"] = np.where(pd.notna(dur), np.round(dur), np.nan)
else:
    df["duration_min"] = np.nan

df.head(10)


,icao24,firstseen,lastseen,callsign,estdepartureairport,estarrivalairport,model,typecode,registration,departure_time,arrival_time,duration_min
0,44cdc6,1.662054e+09,1662065618,BEL40V,None,EBBR,A320 214,A320,OO-SNF,2022-09-01 17:37:49+00:00,2022-09-01 20:53:38+00:00,196.0
1,44cdc6,1.662032e+09,1662047517,BEL7QJ,EBBR,None,A320 214,A320,OO-SNF,2022-09-01 11:41:34+00:00,2022-09-01 15:51:57+00:00,250.0
2,44cdc6,1.662022e+09,1662028405,BEL8DG,LIRF,EBBR,A320 214,A320,OO-SNF,2022-09-01 08:39:54+00:00,2022-09-01 10:33:25+00:00,114.0
3,44cdc6,1.662011e+09,1662016737,BEL1YC,EBBR,LIRF,A320 214,A320,OO-SNF,2022-09-01 05:39:50+00:00,2022-09-01 07:18:57+00:00,99.0
4,4601f7,1.662067e+09,1662068749,FIN7A,EFHK,EFTP,ATR 72 500,AT75,OH-ATJ,2022-09-01 21:14:02+00:00,2022-09-01 21:45:49+00:00,32.0
5,4601f7,1.662059e+09,1662060352,FIN5AZ,EETN,EFHK,ATR 72 500,AT75,OH-ATJ,2022-09-01 19:03:31+00:00,2022-09-01 19:25:52+00:00,22.0
6,4601f7,1.662056e+09,1662057089,FIN9D,EFHK,EETN,ATR 72 500,AT75,OH-ATJ,2022-09-01 18:08:00+00:00,2022-09-01 18:31:29+00:00,23.0
7,4601f7,1.662052e+09,1662053667,FIN27H,EETN,EFHK,ATR 72 500,AT75,OH-ATJ,2022-09-01 17:11:24+00:00,2022-09-01 17:34:27+00:00,23.0
8,4601f7,1.662050e+09,1662050774,FIN5Y,EFHK,EETN,ATR 72 500,AT75,OH-ATJ,2022-09-01 16:27:13+00:00,2022-09-01 16:46:14+00:00,19.0
9,4601f7,1.662046e+09,1662047514,FIN3ZP,None,EFHK,ATR 72 500,AT75,OH-ATJ,2022-09-01 15:21:20+00:00,2022-09-01 15:51:54+00:00,31.0


## 7) Drop unusable rows (critical nulls)

In [41]:
# A row is valid only if all criticals exist & airports are valid ICAO
crit_ok = (
    df["icao24"].notna() &
    df["callsign"].apply(lambda s: isinstance(s, str) and len(s) > 0) &
    df["estdepartureairport"].notna() &
    df["estarrivalairport"].notna() &
    df["departure_time"].notna() &
    df["arrival_time"].notna()
)

df_clean = df.loc[crit_ok].copy()

# Derive ISO ingest date (string)
df_clean["ingest_date"] = pd.to_datetime(df_clean["departure_time"]).dt.date.astype(str)

print("Rows before:", len(df))
print("Rows after cleaning:", len(df_clean))
print("Any invalid dep ICAO? ->", df_clean["estdepartureairport"].str.fullmatch(ICAO4).isna().any())
print("Any invalid arr ICAO? ->", df_clean["estarrivalairport"].str.fullmatch(ICAO4).isna().any())


Rows before: 951916
Rows after cleaning: 610535
Any invalid dep ICAO? -> False
Any invalid arr ICAO? -> False


## 8) Basic QA checks

In [42]:
# Ensure no bad airport codes slipped through
bad_dep = df_clean.loc[~df_clean["estdepartureairport"].str.fullmatch(ICAO4)]
bad_arr = df_clean.loc[~df_clean["estarrivalairport"].str.fullmatch(ICAO4)]

print("Bad dep rows:", len(bad_dep))
print("Bad arr rows:", len(bad_arr))

# Make sure we didn't accidentally create 'NAN'
print("Contains literal 'NAN' in dep?", (df_clean["estdepartureairport"] == "NAN").any())
print("Contains literal 'NAN' in arr?", (df_clean["estarrivalairport"] == "NAN").any())

df_clean.head(5)


Bad dep rows: 0
Bad arr rows: 0
Contains literal 'NAN' in dep? False
Contains literal 'NAN' in arr? False


,icao24,firstseen,lastseen,callsign,estdepartureairport,estarrivalairport,model,typecode,registration,departure_time,arrival_time,duration_min,ingest_date
2,44cdc6,1.662022e+09,1662028405,BEL8DG,LIRF,EBBR,A320 214,A320,OO-SNF,2022-09-01 08:39:54+00:00,2022-09-01 10:33:25+00:00,114.0,2022-09-01
3,44cdc6,1.662011e+09,1662016737,BEL1YC,EBBR,LIRF,A320 214,A320,OO-SNF,2022-09-01 05:39:50+00:00,2022-09-01 07:18:57+00:00,99.0,2022-09-01
4,4601f7,1.662067e+09,1662068749,FIN7A,EFHK,EFTP,ATR 72 500,AT75,OH-ATJ,2022-09-01 21:14:02+00:00,2022-09-01 21:45:49+00:00,32.0,2022-09-01
5,4601f7,1.662059e+09,1662060352,FIN5AZ,EETN,EFHK,ATR 72 500,AT75,OH-ATJ,2022-09-01 19:03:31+00:00,2022-09-01 19:25:52+00:00,22.0,2022-09-01
6,4601f7,1.662056e+09,1662057089,FIN9D,EFHK,EETN,ATR 72 500,AT75,OH-ATJ,2022-09-01 18:08:00+00:00,2022-09-01 18:31:29+00:00,23.0,2022-09-01


## 9) Preview final document structure

In [43]:
import json
preview_docs = df_clean.head(5).to_dict(orient="records")
print(json.dumps(preview_docs, indent=2, default=str))

[
  {
    "icao24": "44cdc6",
    "firstseen": 1662021594.0,
    "lastseen": 1662028405,
    "callsign": "BEL8DG",
    "estdepartureairport": "LIRF",
    "estarrivalairport": "EBBR",
    "model": "A320 214",
    "typecode": "A320",
    "registration": "OO-SNF",
    "departure_time": "2022-09-01 08:39:54+00:00",
    "arrival_time": "2022-09-01 10:33:25+00:00",
    "duration_min": 114.0,
    "ingest_date": "2022-09-01"
  },
  {
    "icao24": "44cdc6",
    "firstseen": 1662010790.0,
    "lastseen": 1662016737,
    "callsign": "BEL1YC",
    "estdepartureairport": "EBBR",
    "estarrivalairport": "LIRF",
    "model": "A320 214",
    "typecode": "A320",
    "registration": "OO-SNF",
    "departure_time": "2022-09-01 05:39:50+00:00",
    "arrival_time": "2022-09-01 07:18:57+00:00",
    "duration_min": 99.0,
    "ingest_date": "2022-09-01"
  },
  {
    "icao24": "4601f7",
    "firstseen": 1662066842.0,
    "lastseen": 1662068749,
    "callsign": "FIN7A",
    "estdepartureairport": "EFHK",
    

## 10) Export JSONL for flights(ensure ints & ISO strings)

In [44]:
from pathlib import Path
import json
import math
import numpy as np

def none_if_nan(v):
    try:
        # treat pandas/NumPy NaN as None
        if v is None:
            return None
        if isinstance(v, float) and math.isnan(v):
            return None
        if isinstance(v, (np.floating,)) and np.isnan(v):
            return None
    except Exception:
        pass
    return v

def prune_nulls(d):
    # drop keys that are None (OK if your validator does not require them)
    return {k: v for k, v in d.items() if v is not None}

OUT_DIR.mkdir(exist_ok=True, parents=True)
out_flights = OUT_DIR / "flights_clean_10d.jsonl"

with out_flights.open("w", encoding="utf-8") as f:
    for rec in df_clean.to_dict(orient="records"):
        # datetimes to ISO
        rec["departure_time"] = rec["departure_time"].isoformat()
        rec["arrival_time"]   = rec["arrival_time"].isoformat()

        # duration as int or None
        rec["duration_min"] = None if pd.isna(rec.get("duration_min")) else int(rec["duration_min"])

        # 🚫 make sure these are either string or None
        for k in ["model", "typecode", "registration"]:
            rec[k] = none_if_nan(rec.get(k))
            # if you prefer to DROP absent fields instead of keeping nulls:
            # if rec[k] is None: rec.pop(k, None)

        # if you prefer pruned docs (no null keys), uncomment:
        # rec = prune_nulls(rec)

        f.write(json.dumps(rec) + "\n")

print("Wrote:", out_flights)


Wrote: ../data_enriched/flights_clean_10d.jsonl


## 11) Export JSONL — Aircraft rollup (optional prune nulls)

In [45]:
# Roll up last known attributes per icao24
ac = df_clean.groupby("icao24").agg({
    "registration":"last", "model":"last", "typecode":"last", "arrival_time":"max"
}).reset_index().rename(columns={"arrival_time":"last_seen"})

# last_seen -> ISO string (or keep None)
ac["last_seen"] = ac["last_seen"].astype(str)

def prune_nulls(d):
    return {k: v for k, v in d.items() if v is not None and str(v) not in ("NaT","None")}

out_ac = OUT_DIR / "aircraft_10d.jsonl"
with out_ac.open("w", encoding="utf-8") as f:
    for rec in ac.to_dict(orient="records"):
        base = {"icao24": rec["icao24"]}
        payload = prune_nulls({k:v for k,v in rec.items() if k!="icao24"})
        f.write(json.dumps({**base, **payload}) + "\n")

print("Wrote:", out_ac)


Wrote: ../data_enriched/aircraft_10d.jsonl


## 12) Next steps

1. Run your database setup scripts:
   ```bash
   mongosh db/01_create_db_and_validation.js
   mongosh db/02_create_indexes.js
   ```
2. Import via PyMongo loader to ensure Date types:
   ```bash
   python db/load_jsonl.py
   ```
3. Verify with sample queries:
   ```bash
   mongosh db/03_sample_crud_and_queries.js
   ```

If you'd like, copy the final logic from sections 6–11 into `etl/etl_flights.py` for production runs.
